In [10]:
import pandas as pd
import numpy as np

import math,datetime
import time

import matplotlib.pyplot as plt
from matplotlib import style

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import model_selection
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import GRU, LSTM, SimpleRNN, Dense
from keras.utils import plot_model
from sklearn.metrics import mean_squared_error
from tensorflow.keras.utils import set_random_seed
from tensorflow.keras import losses, optimizers
from keras.layers import Dense, Dropout, LSTM
from tensorflow.keras import losses, optimizers
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf

In [3]:
pip install yfinance

In [4]:
# Set the seed to generate the same sequence of randomness during multiple executions of the notebook.
seed = 35
tf.random.set_seed(seed)

# Load Amazon stock data from Yahoo Finance from 2001-01-02 to 2024-07-08
ticker = "AMZN"
df = yf.download(ticker, start="2001-01-02", end="2024-07-08")

# Drop any rows with NaN values
df.dropna(inplace=True)

# Preprocess the data by min-max normalization
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))

# Create the time series data structure
look_back = 30  # Look back period is one month.

# Use 80% of data for training and 20% for testing
train_size = int(len(scaled_data) * 0.80)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size - look_back:]

# Generate training and testing sequences of data
train_generator = TimeseriesGenerator(train_data, train_data,
                                     length=look_back, batch_size=20)
test_generator = TimeseriesGenerator(test_data, test_data,
                                     length=look_back, batch_size=1)

[*********************100%%**********************]  1 of 1 completed


In [5]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2001-01-02,0.790500,0.800000,0.681250,0.693750,0.693750,184070000
2001-01-03,0.681250,0.893750,0.681250,0.893750,0.893750,293620000
2001-01-04,0.878125,0.878125,0.771875,0.775000,0.775000,212424000
2001-01-05,0.775000,0.793750,0.678150,0.728150,0.728150,175936000
2001-01-08,0.750000,0.778125,0.687500,0.746875,0.746875,201548000
...,...,...,...,...,...,...
2024-06-28,197.729996,198.850006,192.500000,193.250000,193.250000,76930192
2024-07-01,193.490005,198.295700,192.820007,197.199997,197.199997,41192011
2024-07-02,197.279999,200.429993,195.929993,200.000000,200.000000,45600013


In [8]:
# Building the LSTM model

lstm_model = Sequential()
lstm_model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(look_back, 1)))
lstm_model.add(Dropout(0.2))

lstm_model.add(LSTM(units=70, activation='relu', return_sequences=True))
lstm_model.add(Dropout(0.3))

lstm_model.add(LSTM(units=90, activation='relu', return_sequences=False))
lstm_model.add(Dropout(0.3))

lstm_model.add(Dense(units=1))

# Compile the model
lstm_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [11]:
# Adding early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Fitting the model
history = lstm_model.fit(train_generator, epochs=100, validation_data=test_generator, callbacks=[early_stopping])


Epoch 1/100
235/235 [==============================] - 41s 156ms/step - loss: 0.0067 - mae: 0.0473 - val_loss: 0.0031 - val_mae: 0.0434
Epoch 2/100
235/235 [==============================] - 30s 127ms/step - loss: 5.9507e-04 - mae: 0.0145 - val_loss: 0.0037 - val_mae: 0.0469
Epoch 3/100
235/235 [==============================] - 35s 148ms/step - loss: 0.0018 - mae: 0.0265 - val_loss: 0.0063 - val_mae: 0.0687
Epoch 4/100
235/235 [==============================] - 31s 134ms/step - loss: 0.0011 - mae: 0.0207 - val_loss: 0.0020 - val_mae: 0.0334
Epoch 5/100
235/235 [==============================] - 29s 124ms/step - loss: 6.6228e-04 - mae: 0.0164 - val_loss: 0.0064 - val_mae: 0.0704
Epoch 6/100
235/235 [==============================] - 36s 153ms/step - loss: 7.1478e-04 - mae: 0.0163 - val_loss: 0.0043 - val_mae: 0.0553
Epoch 7/100
235/235 [==============================] - 34s 144ms/step - loss: 4.5539e-04 - mae: 0.0131 - val_loss: 0.0276 - val_mae: 0.1567
Epoch 8/100
235/235 [===========

In [ ]:
# Plotting the training history
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model Loss and MAE')
plt.xlabel('Epoch')
plt.ylabel('Loss/MAE')
plt.legend()
plt.show()

In [ ]:
# Evaluate the model on the test data
loss, mae = lstm_model.evaluate(test_generator)
print(f'Test Loss: {loss}, Test MAE: {mae}')


In [ ]:
# Make predictions
predictions = lstm_model.predict(test_generator)
predictions = scaler.inverse_transform(predictions)



In [ ]:
# Plot the predictions
test_data = df['Close'][train_size:].values[look_back:]
plt.figure(figsize=(10, 6))
plt.plot(test_data, color='blue', label='Actual Stock Price')
plt.plot(predictions, color='red', label='Predicted Stock Price')
plt.title('Amazon Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()